In [1]:
import sys 
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install tensorflow==1.15
!{sys.executable} -m pip install pytesseract
!{sys.executable} -m pip install shapely
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install xlrd

  Using cached https://files.pythonhosted.org/packages/b0/16/63576a1a001752e34bf8ea62e367997530dc553b689356b9879339cf45a4/xlrd-1.2.0-py2.py3-none-any.whl


In [2]:
#imports
import cv2
import time
import math
import os
import numpy as np
import tensorflow as tf
import pytesseract
from pandas import ExcelFile

import locality_aware_nms as nms_locality
import lanms

tf.app.flags.DEFINE_string('test_data_path', 'images/', '')
tf.app.flags.DEFINE_string('gpu_list', '0', '')
tf.app.flags.DEFINE_string('checkpoint_path', 'east_icdar2015_resnet_v1_50_rbox/', '')
tf.app.flags.DEFINE_string('output_dir', 'tmp/', '')
tf.app.flags.DEFINE_string('truth_path', 'W2_Truth_and_Noise_DataSet.xlsx', '')
tf.app.flags.DEFINE_bool('no_write_images', False, 'do not write images')
tf.app.flags.DEFINE_string('f', '', 'kernel')

import model
from icdar import restore_rectangle

FLAGS = tf.app.flags.FLAGS

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
#unchanged
def get_images():
    '''
    find image files in test data path
    :return: list of files found
    '''
    files = []
    exts = ['jpg', 'png', 'jpeg', 'JPG']
    for parent, dirnames, filenames in os.walk(FLAGS.test_data_path):
        for filename in filenames:
            for ext in exts:
                if filename.endswith(ext):
                    files.append(os.path.join(parent, filename))
                    break
    print('Find {} images'.format(len(files)))
    return files

#unchanged
def resize_image(im, max_side_len=2400):
    '''
    resize image to a size multiple of 32 which is required by the network
    :param im: the resized image
    :param max_side_len: limit of max image size to avoid out of memory in gpu
    :return: the resized image and the resize ratio
    '''
    h, w, _ = im.shape

    resize_w = w
    resize_h = h

    # limit the max side
    if max(resize_h, resize_w) > max_side_len:
        ratio = float(max_side_len) / resize_h if resize_h > resize_w else float(max_side_len) / resize_w
    else:
        ratio = 1.
    resize_h = int(resize_h * ratio)
    resize_w = int(resize_w * ratio)

    resize_h = resize_h if resize_h % 32 == 0 else (resize_h // 32 - 1) * 32
    resize_w = resize_w if resize_w % 32 == 0 else (resize_w // 32 - 1) * 32
    resize_h = max(32, resize_h)
    resize_w = max(32, resize_w)
    im = cv2.resize(im, (int(resize_w), int(resize_h)))

    ratio_h = resize_h / float(h)
    ratio_w = resize_w / float(w)

    return im, (ratio_h, ratio_w)
#unchanged
def detect(score_map, geo_map, timer, score_map_thresh=0.8, box_thresh=0.1, nms_thres=0.2):
    '''
    restore text boxes from score map and geo map
    :param score_map:
    :param geo_map:
    :param timer:
    :param score_map_thresh: threshhold for score map
    :param box_thresh: threshhold for boxes
    :param nms_thres: threshold for nms
    :return:
    '''
    if len(score_map.shape) == 4:
        score_map = score_map[0, :, :, 0]
        geo_map = geo_map[0, :, :, ]
    # filter the score map
    xy_text = np.argwhere(score_map > score_map_thresh)
    # sort the text boxes via the y axis
    xy_text = xy_text[np.argsort(xy_text[:, 0])]
    # restore
    start = time.time()
    text_box_restored = restore_rectangle(xy_text[:, ::-1]*4, geo_map[xy_text[:, 0], xy_text[:, 1], :]) # N*4*2
    print('{} text boxes before nms'.format(text_box_restored.shape[0]))
    boxes = np.zeros((text_box_restored.shape[0], 9), dtype=np.float32)
    boxes[:, :8] = text_box_restored.reshape((-1, 8))
    boxes[:, 8] = score_map[xy_text[:, 0], xy_text[:, 1]]
    timer['restore'] = time.time() - start
    # nms part
    start = time.time()
    # boxes = nms_locality.nms_locality(boxes.astype(np.float64), nms_thres)
    boxes = lanms.merge_quadrangle_n9(boxes.astype('float32'), nms_thres)
    timer['nms'] = time.time() - start

    if boxes.shape[0] == 0:
        return None, timer

    # here we filter some low score boxes by the average score map, this is different from the orginal paper
    for i, box in enumerate(boxes):
        mask = np.zeros_like(score_map, dtype=np.uint8)
        cv2.fillPoly(mask, box[:8].reshape((-1, 4, 2)).astype(np.int32) // 4, 1)
        boxes[i, 8] = cv2.mean(score_map, mask)[0]
    boxes = boxes[boxes[:, 8] > box_thresh]

    return boxes, timer
#unchanged
def sort_poly(p):
    min_axis = np.argmin(np.sum(p, axis=1))
    p = p[[min_axis, (min_axis+1)%4, (min_axis+2)%4, (min_axis+3)%4]]
    if abs(p[0, 0] - p[1, 0]) > abs(p[0, 1] - p[1, 1]):
        return p
    else:
        return p[[0, 3, 2, 1]]

In [4]:
# convert excel into readable pandas format and then convert to python dict

def get_excel_docs(excel_path:str, sheet:int):
    xls = ExcelFile(excel_path)

    # have two dataframes, one that is used to get the correct file names and one for the truth
    file_names_df = xls.parse(xls.sheet_names[sheet])
    truth_df = xls.parse(xls.sheet_names[0])

    file_name_docs = file_names_df.to_dict('records')
    truth_docs = truth_df.to_dict('records')

    # get the file names from the specified sheet
    truth_file_name_list = []
    if sheet == 0:
        for doc in file_name_docs:
            truth_file_name_list.append(doc['File_BaseName'])
    elif sheet == 1:
        for doc in file_name_docs:
            truth_file_name_list.append(doc['file_name'])
    else:
        raise ValueError("Invalid sheet number")
    return truth_file_name_list, truth_docs

def typeFloat(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [5]:
#get boxes
def getBox(box,im):
    h = max(box[2,1],box[3,1]) - min(box[0,1],box[1,1])
    w = max(box[1,0],box[2,0]) - min(box[0,0],box[3,0])
    x = min(box[0,0],box[3,0])
    y = min(box[0,1],box[1,1])
    crop_img = im[y:y + h, x:x + w]
    return crop_img

In [6]:
def main(argv=None):
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu_list


    try:
        os.makedirs(FLAGS.output_dir)
    except OSError as e:
        if e.errno != 17:
            raise

    with tf.get_default_graph().as_default():
        input_images = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_images')
        global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)

        f_score, f_geometry = model.model(input_images, is_training=False)

        variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
        saver = tf.train.Saver(variable_averages.variables_to_restore())

        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            ckpt_state = tf.train.get_checkpoint_state(FLAGS.checkpoint_path)
            model_path = os.path.join(FLAGS.checkpoint_path, os.path.basename(ckpt_state.model_checkpoint_path))
            print('Restore from {}'.format(model_path))
            saver.restore(sess, model_path)
            parse = []
            im_fn_list = get_images()
            dir = '/Users/Taaha/Documents/projects/EAST'
            excel_path = os.path.join(dir, FLAGS.truth_path)
            truth_file_name_list, truth_docs = get_excel_docs(excel_path, 0)
            

            # Open text file with headers
            with open ("headers.txt", "r") as myFile : 
                headers = myFile.read().split()

            for index, im_fn in enumerate(im_fn_list):
                # save accuracy and time to compute averages
                accuracy_list = []
                time_list = []
                float_accuracy_list = []
                string_accuracy_list = []
                fuzzy_score_list = []

                floatCorrect = 0
                floatWrong = 0
                stringCorrect = 0
                stringWrong = 0
                num_correct = 0
                num_total = 0
                allWords = []
                
                im = cv2.imread(im_fn)[:, :, ::-1]
                start_time = time.time()
                im_resized, (ratio_h, ratio_w) = resize_image(im)

                timer = {'net': 0, 'restore': 0, 'nms': 0}
                start = time.time()
                score, geometry = sess.run([f_score, f_geometry], feed_dict={input_images: [im_resized]})
                timer['net'] = time.time() - start

                boxes, timer = detect(score_map=score, geo_map=geometry, timer=timer)
                print('{} : net {:.0f}ms, restore {:.0f}ms, nms {:.0f}ms'.format(
                    im_fn, timer['net']*1000, timer['restore']*1000, timer['nms']*1000))

                if boxes is not None:
                    boxes = boxes[:, :8].reshape((-1, 4, 2))
                    boxes[:, :, 0] /= ratio_w
                    boxes[:, :, 1] /= ratio_h

                duration = time.time() - start_time
                print('[timing] {}'.format(duration))

                #truth
                truth_doc = truth_docs[index]
                field_names = truth_doc.values()
                start_time = time.time()

                # save to file
                if boxes is not None:
                    res_file = os.path.join(
                        FLAGS.output_dir,
                        '{}.txt'.format(
                            os.path.basename(im_fn).split('.')[0]))
                    with open(res_file, 'w') as f:
                        for box in boxes:
                            # to avoid submitting errors
                            box = sort_poly(box.astype(np.int32))
                            if np.linalg.norm(box[0] - box[1]) < 5 or np.linalg.norm(box[3]-box[0]) < 5:
                                continue
                            f.write('{},{},{},{},{},{},{},{}\r\n'.format(
                                box[0, 0], box[0, 1], box[1, 0], box[1, 1], box[2, 0], box[2, 1], box[3, 0], box[3, 1],
                            ))
                            
                            crop_img = getBox(box,im)
                            output = pytesseract.image_to_string(crop_img)
                            allWords.append(output)

                            #cv2.imshow("box",crop_img)
                            #cv2.waitKey(0)
                            cv2.polylines(im[:, :, ::-1], [box.astype(np.int32).reshape((-1, 1, 2))], True, color=(255, 255, 0), thickness=1)
                
                end_time = time.time()
                allWords.remove('')
                parse = ' '.join(allWords)

                for field_name in field_names:
                    if str(field_name) in parse:
                        num_correct += 1
                        if(typeFloat(str(field_name))):
                            floatCorrect += 1

                        else:
                            stringCorrect += 1
                            parse.replace(str(field_name),'',1)
                    else:
                        if(typeFloat(str(field_name))):
                            floatWrong += 1
                        else:
                            stringWrong += 1
                    num_total += 1

                                # code to check for headers
                for heading in headers:
                    if str(heading) in parse:
                        num_correct += 1
                        if (typeFloat(str(heading))):
                            floatCorrect += 1

                        else: 
                            stringCorrect += 1
                        parse.replace(str(heading), '', 1)

                    else: 
                        if (typeFloat(str(heading))):
                            floatWrong += 1
                        else:
                            stringWrong += 1
                    num_total += 1

                accuracy = (num_correct / num_total) * 100
                time_spent = end_time - start_time
                floatAccuracy = (floatCorrect / (floatCorrect+floatWrong)) * 100
                stringAccuracy = (stringCorrect / (stringCorrect + stringWrong)) * 100
                #print(doc_name)
                print("Accuracy", accuracy)
                print("float Accuracy", floatAccuracy)
                print("String Accuracy", stringAccuracy)
                print("Time to parse document: {} seconds".format(time_spent))
                print("==================================================================")
                accuracy_list.append(accuracy)
                time_list.append(time_spent)
                float_accuracy_list.append(floatAccuracy)
                string_accuracy_list.append(stringAccuracy)

                if not FLAGS.no_write_images:
                    img_path = os.path.join(FLAGS.output_dir, os.path.basename(im_fn))
                    cv2.imwrite(img_path, im[:, :, ::-1])
            

In [7]:
main()
#if __name__ == '__main__':
 #   tf.app.run()
    




Instructions for updating:
Please use `layer.__call__` method instead.
resnet_v1_50/block1 (?, ?, ?, 256)
resnet_v1_50/block2 (?, ?, ?, 512)
resnet_v1_50/block3 (?, ?, ?, 1024)
resnet_v1_50/block4 (?, ?, ?, 2048)
Shape of f_0 (?, ?, ?, 2048)
Shape of f_1 (?, ?, ?, 512)
Shape of f_2 (?, ?, ?, 256)
Shape of f_3 (?, ?, ?, 64)

Shape of h_0 (?, ?, ?, 2048), g_0 (?, ?, ?, 2048)
Shape of h_1 (?, ?, ?, 128), g_1 (?, ?, ?, 128)
Shape of h_2 (?, ?, ?, 64), g_2 (?, ?, ?, 64)
Shape of h_3 (?, ?, ?, 32), g_3 (?, ?, ?, 32)
Restore from east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491
INFO:tensorflow:Restoring parameters from east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491
Find 3 images
7404 text boxes before nms
images/W2_XL_input_noisy_1000.jpg : net 2139ms, restore 38ms, nms 2239ms
[timing] 4.496397256851196
Accuracy 57.95454545454546
float Accuracy 34.78260869565217
String Accuracy 66.15384615384615
Time to parse document: 69.3911919593811 seconds
7103 text boxes before nms
images/W2_XL_inp

In [1]:
!pip freeze

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
altgraph==0.10.2
appdirs==1.4.3
appnope==0.1.0
asn1crypto==0.24.0
attrs==19.3.0
backports-abc==0.5
backports.shutil-get-terminal-size==1.0.0
bdist-mpkg==0.5.0
bleach==3.1.4
bonjour-py==0.3
cffi==1.12.2
configparser==4.0.2
contextlib2==0.6.0.post1
cryptography==2.6.1
decorator==4.4.2
defusedxml==0.6.0
distlib==0.3.0
entrypoints==0.3
enum34==1.1.6
filelock==3.0.12
functools32==3.2.3.post2
future==0.17.1
futures==3.3.0
graphviz==0.13.2
importlib-metadata==1.5.0
importlib-resources==1.2.0
ipaddress==1.0.22
ipykernel==4.10.1
ipython==5.9.0
ipython-genutils==0.2.0
Jinja2==2.11.2
jsonschema==3.2.0
jupyter-client==5.3.4
jupyter-core==4.6.3
jupyterlab==0.33.12
jupyte